In [ ]:
import sys
sys.path.append("./notebooks")
sys.path.append("../src")

import matplotlib.pyplot as plt
import numpy as np
from data_processing.geometry_processor import GeometryProcessor
from data_processing.band_dto import BandDTO
from data_processing.feature_service import FeatureService
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from scipy.stats import gaussian_kde

In [ ]:
gp = GeometryProcessor()
band_data  = gp.flatten_and_filter_monthly_data()

In [ ]:
fs = FeatureService(band_data)
feature_df = fs.calculate_features_for_monthly_data()

feature_df.to_csv("features.csv")

In [ ]:
scaler = MinMaxScaler()
feature_norm = scaler.fit_transform(feature_df)

pca = PCA(n_components=3)
X_transformed = pca.fit_transform(feature_norm)

In [ ]:
plot_data = X_transformed[::50].T
kde = gaussian_kde(plot_data)
density = kde(plot_data)

fig = go.Figure(data=[go.Scatter3d(
    x=X_transformed[::50, 0],
    y=X_transformed[::50, 1],
    z=X_transformed[::50, 2],
    mode='markers',
    marker=dict(
        size=4,
        color=density,
        colorscale='Viridis',
        opacity=0.8,
        colorbar=dict(title='Density')
    )
)])

fig.update_layout(
    title='3D Density Scatter Plot',
    scene=dict(
        xaxis_title='NDRE783',
        yaxis_title='B11',
        zaxis_title='WRI'
    ),
    margin=dict(l=0, r=0, b=20, t=40),
    width=800,
    height=600
)

fig.show()